## Ref

+ Dynamic Matrix Factorization with Priors on Unknown Values
+ https://arxiv.org/pdf/1507.06452.pdf

### imports

In [1]:
import numpy as np
import csv
import random
import time
import copy
import pickle

### Load data

In [12]:
train = []
update = []
test = []

with open("yelp_online_train.csv", newline = "") as f:
    rows = csv.reader(f)
    for row in rows:
        if row[0] == "user_id":
            continue
        train.append(row)
        
with open("yelp_online_test.csv", newline = "") as f:
    rows = csv.reader(f)
    for row in rows:
        if row[0] == "user_id":
            continue
        test.append(row)

In [13]:
train[:5]

[['14445', '359', '20041019'],
 ['14445', '18677', '20041019'],
 ['14445', '8611', '20041019'],
 ['17457', '5223', '20050303'],
 ['17457', '17278', '20050303']]

In [14]:
test[:5]

[['10137', '16510', '20140710'],
 ['22234', '17678', '20140710'],
 ['17402', '6048', '20140710'],
 ['2716', '17592', '20140710'],
 ['18382', '14453', '20140710']]

In [15]:
user_dict = {}
item_dict = {}
user_count = 0
item_count = 0

for row in train:
    if row[0] not in user_dict:
        user_dict[row[0]] = user_count
        user_count += 1
    if row[1] not in item_dict:
        item_dict[row[1]] = item_count
        item_count += 1
        
for row in test:
    if row[0] not in user_dict:
        user_dict[row[0]] = user_count
        user_count += 1
    if row[1] not in item_dict:
        item_dict[row[1]] = item_count
        item_count += 1

In [16]:
user_dict_2 = {}
item_dict_2 = {}

for user in user_dict:
    user_dict_2[user_dict[user]] = user
for item in item_dict:
    item_dict_2[item_dict[item]] = item

In [17]:
for row in train:
    row[0] = user_dict[row[0]]
    row[1] = item_dict[row[1]]
    
for row in test:
    row[0] = user_dict[row[0]]
    row[1] = item_dict[row[1]]

In [18]:
train[:5]

[[0, 0, '20041019'],
 [0, 1, '20041019'],
 [0, 2, '20041019'],
 [1, 3, '20050303'],
 [1, 4, '20050303']]

In [19]:
test[:5]

[[3964, 2879, '20140710'],
 [26577, 6814, '20140710'],
 [29523, 20791, '20140710'],
 [27992, 2621, '20140710'],
 [19469, 6199, '20140710']]

### data size

In [20]:
n_user = len(user_dict)
n_item = len(item_dict)
n_rating = len(train)

print("number of ratings:", n_rating)
print("n_user:", n_user)
print("n_item:", n_item)

number of ratings: 791010
n_user: 30407
n_item: 22676


### number of training and updating

In [23]:
n_train = len(train)
n_test = len(test)

print("n_train:", n_train)
print("n_test:", n_test)

n_train: 791010
n_test: 87891


### construct rating matrix

In [24]:
ratings = []
zeros = np.zeros(n_item)

for _ in range(n_user):
    ratings.append(zeros.copy())
    
for i in range(n_train):
    ratings[train[i][0]][train[i][1]] = 1.0

In [25]:
ratings[0][:10]

array([ 1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])

### list rated movies for each user & for each movie

In [26]:
rate_lists_user = []
rate_lists_item = []

for i in range(n_user):
    tmp = []
    for j in range(n_item):
        if ratings[i][j] > 0.1:
            tmp.append(j)
    rate_lists_user.append(tmp.copy())
    
for i in range(n_item):
    tmp = []
    for j in range(n_user):
        if ratings[j][i] > 0.1:
            tmp.append(j)
    rate_lists_item.append(tmp.copy())

In [27]:
len(rate_lists_user[0])

23

In [28]:
len(rate_lists_item[0])

618

### initialize user and item vectors

In [29]:
user_vecs = []
item_vecs = []
dim = 100

def noise(dim):
    return np.random.uniform(-1, 1, dim)

for i in range(n_user):
    user_vecs.append(noise(dim))
for i in range(n_item):
    item_vecs.append(noise(dim))

In [30]:
user_vecs[0]

array([-0.65902836, -0.34008991,  0.65085957, -0.45834596, -0.92854269,
       -0.98596853,  0.06566744, -0.46271555,  0.53410863,  0.50988186,
       -0.34534047, -0.47128739,  0.67343248,  0.0317375 , -0.07125827,
       -0.84815636, -0.66483843, -0.0717082 , -0.27068584, -0.70741081,
        0.88943354, -0.57062524,  0.96252919, -0.79113895, -0.04980071,
        0.93659021, -0.60635948, -0.16282006, -0.04278514,  0.03294168,
       -0.44380625, -0.20815702,  0.23547242,  0.44349931,  0.11999413,
       -0.63493331, -0.79433162,  0.00274636,  0.61953007, -0.71072731,
       -0.54541859, -0.82116014, -0.14800144,  0.80282859, -0.74737566,
        0.59998118, -0.06525424, -0.208489  , -0.2229839 , -0.60835033,
       -0.17941261,  0.84495504,  0.58211732,  0.00337052, -0.30166136,
       -0.81227672, -0.40561745, -0.88715046, -0.32770136, -0.56694948,
        0.1217122 ,  0.47963071,  0.04400264,  0.13790101,  0.25048949,
       -0.3685431 , -0.23893675, -0.39239855, -0.9123951 , -0.45

### calculate $S^{user}$ and $S^{item}$

In [31]:
def vec_mul(arr):
    l = len(arr)
    ret = []
    
    for i in range(l):
        tmp = np.zeros(l)
        for j in range(l):
            tmp[j] = arr[i] * arr[j]
        ret.append(tmp)
        
    return ret

In [32]:
def update_s_user():
    for i in range(n_user):
        s = vec_mul(user_vecs[i])
        for j in range(dim):
            s_user[j] += s[j]

def update_s_item():
    for i in range(n_item):
        s = vec_mul(item_vecs[i])
        for j in range(dim):
            s_item[j] += s[j]

### gradient of loss

In [33]:
def dot(a, b):
    ret = 0.0
    for i in range(len(a)):
        ret += a[i] * b[i]
    return ret

def sign(x):
    if x >= 0:
        return 1
    else:
        return -1

def gradient_user(i, alpha, lmd):
    ret = np.zeros(dim)
    # 2 * \alpha * w_i * S^h
    for j in range(dim):
        ret[j] += 2 * alpha * dot(user_vecs[i], s_item[j])
    # \lambda * sign(w_i)
    for j in range(dim):
        ret[j] += lmd * sign(user_vecs[i][j])
    # -2 * \sum{h_j}
    for j in range(len(rate_lists_user[i])):
        no = rate_lists_user[i][j]
        rate = ratings[i][no]
        scalar = rate - (1-alpha) * dot(user_vecs[i], item_vecs[no])
        scalar *= -2
        ret += scalar * item_vecs[no]
    return ret

def gradient_item(i, alpha, lmd):
    ret = np.zeros(dim)
    # 2 * \alpha * h_i * S^w
    for j in range(dim):
        ret[j] += 2 * alpha * dot(item_vecs[i], s_user[j])
    # \lambda * sign(h_i)
    for j in range(dim):
        ret[j] += lmd * sign(item_vecs[i][j])
    # -2 * \sum{w_j}
    for j in range(len(rate_lists_item[i])):
        no = rate_lists_item[i][j]
        rate = ratings[no][i]
        scalar = rate - (1-alpha) * dot(item_vecs[i], user_vecs[no])
        scalar *= -2
        ret += scalar * user_vecs[no]
    return ret

def gradient_user_u(i, alpha, lmd):
    ret = np.zeros(dim)
    # 2 * \alpha * w_i * S^h
    for j in range(dim):
        ret[j] += 2 * alpha * dot(user_vecs_u[i], s_item_u[j])
    # \lambda * sign(w_i)
    for j in range(dim):
        ret[j] += lmd * sign(user_vecs_u[i][j])
    # -2 * \sum{h_j}
    for j in range(len(rate_lists_user_u[i])):
        no = rate_lists_user_u[i][j]
        rate = ratings_u[i][no]
        scalar = rate - (1-alpha) * dot(user_vecs_u[i], item_vecs_u[no])
        scalar *= -2
        ret += scalar * item_vecs_u[no]
    return ret

def gradient_item_u(i, alpha, lmd):
    ret = np.zeros(dim)
    # 2 * \alpha * h_i * S^w
    for j in range(dim):
        ret[j] += 2 * alpha * dot(item_vecs_u[i], s_user_u[j])
    # \lambda * sign(h_i)
    for j in range(dim):
        ret[j] += lmd * sign(item_vecs_u[i][j])
    # -2 * \sum{w_j}
    for j in range(len(rate_lists_item_u[i])):
        no = rate_lists_item_u[i][j]
        rate = ratings_u[no][i]
        scalar = rate - (1-alpha) * dot(item_vecs_u[i], user_vecs_u[no])
        scalar *= -2
        ret += scalar * user_vecs_u[no]
    return ret

### hyperparameters

In [52]:
lmd = 0.01
alpha = 0.0023
rho = alpha * (n_user*n_item - n_train) / n_train
EPOCH = 30
lr = 0.001 # the paper use line search, but setting learning rate is good enough

print("lambda:", lmd)
print("alpha:", alpha)
print("rho:", rho)
print("learning rate:", lr)

lambda: 0.01
alpha: 0.0023
rho: 2.002568463862657
learning rate: 0.001


### train

In [40]:
def update_user(i, scalar):
    mat = vec_mul(user_vecs[i])
    for j in range(dim):
        s_user[j] += scalar * mat[j]
    return

def update_item(i, scalar):
    mat = vec_mul(item_vecs[i])
    for j in range(dim):
        s_item[j] += scalar * mat[j]
    return

def update_user_u(i, scalar):
    mat = vec_mul(user_vecs_u[i])
    for j in range(dim):
        s_user_u[j] += scalar * mat[j]
    return

def update_item_u(i, scalar):
    mat = vec_mul(item_vecs_u[i])
    for j in range(dim):
        s_item_u[j] += scalar * mat[j]
    return

In [24]:
# def cal_loss():
#     ret = 0.0
#     for i in range(n_user):
#         for j in range(n_item):
#             if ratings[i][j] > 0.1:
#                 residual = ratings[i][j] - dot(user_vecs[i], item_vecs[j])
#                 ret += residual ** 2
#             else:
#                 residual = ratings[i][j] - dot(user_vecs[i], item_vecs[j])
#                 ret += alpha * (residual ** 2)
#     sum_w = 0.0
#     for i in range(n_user):
#         for j in range(dim):
#             sum_w += sign(user_vecs[i][j]) * user_vecs[i][j]
#     for i in range(n_item):
#         for j in range(dim):
#             sum_w += sign(item_vecs[i][j]) * item_vecs[i][j]
#     return ret + lmd * sum_w

# def cal_loss_u():
#     ret = 0.0
#     for i in range(n_user):
#         for j in range(n_item):
#             if ratings_u[i][j] > 0.1:
#                 residual = ratings_u[i][j] - dot(user_vecs_u[i], item_vecs_u[j])
#                 ret += residual ** 2
#             else:
#                 residual = ratings_u[i][j] - dot(user_vecs_u[i], item_vecs_u[j])
#                 ret += alpha * (residual ** 2)
#     sum_w = 0.0
#     for i in range(n_user):
#         for j in range(dim):
#             sum_w += sign(user_vecs_u[i][j]) * user_vecs_u[i][j]
#     for i in range(n_item):
#         for j in range(dim):
#             sum_w += sign(item_vecs_u[i][j]) * item_vecs_u[i][j]
#     return ret + lmd * sum_w

In [42]:
def cal_acc(n):
    predict = []
    rates = []
    error = 0.0

    for i in range(n):
        rates.append(1.0)
        predict.append(dot(user_vecs[test[i][0]], item_vecs[test[i][1]]))

    for i in range(n):
        error += abs(rates[i] - predict[i])**2

    return error / n

def cal_acc_u(n):
    predict = []
    rates = []
    error = 0.0

    for i in range(n):
        rates.append(1.0)
        predict.append(dot(user_vecs_u[test[i][0]], item_vecs_u[test[i][1]]))

    for i in range(n):
        error += abs(rates[i] - predict[i])**2

    return error / n

def cal_insample_acc(n):
    predict = []
    rates = []
    error = 0.0

    for i in range(n):
        rates.append(1.0)
        predict.append(dot(user_vecs[train[i][0]], item_vecs[train[i][1]]))

    for i in range(n):
        error += abs(rates[i] - predict[i])**2

    return error / n

def cal_insample_acc_u(n):
    predict = []
    rates = []
    error = 0.0

    for i in range(n):
        rates.append(1.0)
        predict.append(dot(user_vecs_u[train[i][0]], item_vecs_u[train[i][1]]))

    for i in range(n):
        error += abs(rates[i] - predict[i])**2

    return error / n

In [53]:
user_vecs = []
item_vecs = []
s_user = vec_mul(np.zeros(dim))
s_item = vec_mul(np.zeros(dim))

for i in range(n_user):
    user_vecs.append(noise(dim))
for i in range(n_item):
    item_vecs.append(noise(dim))

update_s_user()
update_s_item()

user_vecs_u = copy.deepcopy(user_vecs)
item_vecs_u = copy.deepcopy(item_vecs)
s_user_u = copy.deepcopy(s_user)
s_item_u = copy.deepcopy(s_item)

In [54]:
user_vecs = copy.deepcopy(user_vecs_u)
item_vecs = copy.deepcopy(item_vecs_u)
s_user = copy.deepcopy(s_user_u)
s_item = copy.deepcopy(s_item_u)

start_train_time = time.time()
cal_loss_time = 0.0
cal_acc_time = 0.0

losses = []
in_accs = []
out_accs = []

In [ ]:
for epoch in range(EPOCH):
    lis_user = list(range(n_user))
    lis_item = list(range(n_item))
    random.shuffle(lis_user)
    random.shuffle(lis_item)
    count = 0
    
    for i in lis_user:
        grad = gradient_user(i, alpha, lmd)
        update_user(i, -1)
        user_vecs[i] -= lr * grad
        update_user(i, 1)
        
        if count % 100 == 99:
            print("user:", count+1, "; in-sample error:", cal_insample_acc(n_test), "; out-sample error", cal_acc(n_test))
        count += 1
    
    count = 0
    
    for i in lis_item:
        grad = gradient_item(i, alpha, lmd)
        update_item(i, -1)
        item_vecs[i] -= lr * grad
        update_item(i, 1)
        
        if count % 100 == 99:
            print("item:", count+1, "; in-sample error:", cal_insample_acc(n_test), "; out-sample error", cal_acc(n_test))
        count += 1
    
    loss_time = time.time()
    # loss = 0
    # loss = cal_loss()
    # losses.append(loss)
    cal_loss_time += time.time() - loss_time
    
    acc_time = time.time()
    in_acc = cal_insample_acc(n_test)
    out_acc = cal_acc(n_test)
    in_accs.append(in_acc)
    out_accs.append(out_acc)
    cal_acc_time += time.time() - acc_time
    
    print("\nepoch", epoch, ": averge error(in):", in_acc, "; averge error(out):", out_acc, "\n")

print("total training time:", time.time()-start_train_time)
print("cal loss time:", cal_loss_time)
print("cal accuracy time:", cal_acc_time)

user: 100 ; in-sample error: 12.1927058178 ; out-sample error 12.1396386526
user: 200 ; in-sample error: 12.1814112375 ; out-sample error 12.1352146654
user: 300 ; in-sample error: 12.1743500093 ; out-sample error 12.1283777134
user: 400 ; in-sample error: 12.1536104585 ; out-sample error 12.1251883898
user: 500 ; in-sample error: 12.1365359672 ; out-sample error 12.1213009113
user: 600 ; in-sample error: 12.1297822877 ; out-sample error 12.1165895075
user: 700 ; in-sample error: 12.11430647 ; out-sample error 12.1119693764
user: 800 ; in-sample error: 12.0855013399 ; out-sample error 12.1065241999
user: 900 ; in-sample error: 12.0729979289 ; out-sample error 12.1012206259
user: 1000 ; in-sample error: 12.0629289719 ; out-sample error 12.0983412395
user: 1100 ; in-sample error: 12.0551423892 ; out-sample error 12.0955123917
user: 1200 ; in-sample error: 12.0448729335 ; out-sample error 12.0925067844
user: 1300 ; in-sample error: 12.0327675696 ; out-sample error 12.0893840751
user: 1400

user: 10800 ; in-sample error: 11.0065937939 ; out-sample error 11.6696097211
user: 10900 ; in-sample error: 10.9999408309 ; out-sample error 11.6672539346
user: 11000 ; in-sample error: 10.9952826112 ; out-sample error 11.6654341838
user: 11100 ; in-sample error: 10.9837943104 ; out-sample error 11.6598314638
user: 11200 ; in-sample error: 10.9748090822 ; out-sample error 11.6550824391
user: 11300 ; in-sample error: 10.9616062604 ; out-sample error 11.6501025185
user: 11400 ; in-sample error: 10.9446257988 ; out-sample error 11.647344335
user: 11500 ; in-sample error: 10.9285351283 ; out-sample error 11.6437599164
user: 11600 ; in-sample error: 10.9034754052 ; out-sample error 11.6404803936
user: 11700 ; in-sample error: 10.8933507987 ; out-sample error 11.6367527458
user: 11800 ; in-sample error: 10.8801040838 ; out-sample error 11.6309691962
user: 11900 ; in-sample error: 10.8623791263 ; out-sample error 11.6279705411
user: 12000 ; in-sample error: 10.85404005 ; out-sample error 11.

### update & predict

In [49]:
user_vecs_u = copy.deepcopy(user_vecs)
item_vecs_u = copy.deepcopy(item_vecs)
s_user_u = copy.deepcopy(s_user)
s_item_u = copy.deepcopy(s_item)
ratings_u = copy.deepcopy(ratings)
rate_lists_user_u = copy.deepcopy(rate_lists_user)
rate_lists_item_u = copy.deepcopy(rate_lists_item)

In [30]:
not_rating_list = []

for i in range(n_user):
    tmp = []
    for j in range(n_item):
        if j not in rate_lists_user_u:
            tmp.append(j)
    not_rating_list.append(tmp.copy())

In [ ]:
def take_value(elem):
    return elem[1]

update_epoch = 100
update_lr = 0.0001
update_time_for_one_print = 0.0
predicts = []
    
for i in range(n_test):
    # predict
    tmp = []
    for j in not_rating_list[test[i][0]]:
        tmp.append([item_dict_2[j], dot(user_vecs_u[test[i][0]], item_vecs_u[j])])
    tmp.sort(key = take_value)

    row = []
    row.append(user_dict_2[test[i][0]]) # user_id
    row.append(test[i][3]) # time

    for j in range(100):
        row.append(tmp[j][0])

    predicts.append(row)

    # update model
    ratings_u[test[i][0]][test[i][1]] = 1.0

    if test[i][1] not in rate_lists_user_u:
        rate_lists_user_u.append(test[i][1])
        not_rating_list[test[i][0]].remove(test[i][1])
    if test[i][0] not in rate_lists_item_u:
        rate_lists_item_u.append(test[i][0])
        
    start_update_time = time.time()

    for epoch in range(update_epoch):
        # update user vector
        grad = gradient_user_u(test[i][0], alpha, lmd)
        update_user_u(test[i][0], -1)
        user_vecs_u[test[i][0]] -= update_lr * grad
        update_user_u(test[i][0], 1)
        # update item vector
        grad = gradient_item_u(test[i][1], alpha, lmd)
        update_item_u(test[i][1], -1)
        item_vecs_u[test[i][1]] -= update_lr * grad
        update_item_u(test[i][1], 1)
        
    update_time_for_one_print += time.time() - start_update_time

    if i % 10 == 9:
        in_acc = cal_insample_acc_u(n_test)
        out_acc = cal_acc_u(n_test)
        in_accs.append(in_acc)
        out_accs.append(out_acc)
        print(i+1, ": averge error(in):", cal_insample_acc_u(1000), "; averge error(out):", cal_acc_u(1000))
        print("time consumed:", update_time_for_one_print)
        update_time_for_one_print = 0.0

In [ ]:
with open("perdict_yelp_online.csv", "w", newline = "") as f:
    w = csv.writer(f)
    for row in predicts:
        w.writerow(row)

### test performance (in-sample after update)

In [ ]:
predict = []
rates = []

# for i in range(n_train):
#     rates.append(1.0)
#     predict.append(dot(user_vecs_u[train[i][0]], item_vecs_u[train[i][1]]))
    
for i in range(n_test):
    rates.append(1.0)
    predict.append(dot(user_vecs_u[test[i][0]], item_vecs_u[test[i][1]]))

In [ ]:
for i in range(30):
    print("ans:", rates[i], "; predict:", predict[i])

In [ ]:
error = 0.0

for i in range(len(rates)):
    error += abs(rates[i] - predict[i])**2
    
print("averge error:", error / len(rates))

### plot

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# plt.plot(losses)
# plt.xlabel("epoch")
# plt.ylabel("loss")
# plt.show()

In [ ]:
plt.plot(in_accs)
plt.plot(out_accs)
plt.xlabel("epoch")
plt.ylabel("error")
plt.show()